In [ ]:
from flask import Flask, jsonify, request
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
from flask import Flask ,request, jsonify
from werkzeug.utils import secure_filename
from flask_cors import CORS
import os
import pymysql
from flask_mysqldb import MySQL
import mysql.connector
import sqlite3
import MySQLdb
import re
import bcrypt
import datetime
from flask_jwt_extended import JWTManager, create_access_token, jwt_required


model = load_model('E:/IIT/py/New folder/model_resnet152v2.h5')
pred_classes = {0:'Anthracnose',
                1:'algal leaf',
                2:'bird eye spot',
                3:'brown blight',
                4:'gray light',
                5:'healthy',
                6:'red leaf spot',
                7:'white spot'}

app = Flask(__name__)
CORS(app)

app.config['MYSQL_HOST'] = "localhost"
app.config['MYSQL_USER'] = "root"
app.config['MYSQL_PASSWORD'] = ""
app.config['MYSQL_DB'] = "tdds"

mysql = MySQL(app)

app.config['JWT_SECRET_KEY'] = 'dnjnv4563kjkk2QDSzkd'
jwt = JWTManager(app)




@app.route('/api/predict', methods=['GET', 'POST'])
def predict():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image uploaded.'})
        img_file = request.files['image']
        img = Image.open(img_file.stream)
        img = img.resize((224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img/255
        pred = model.predict(img)
        pred = pred.argmax(axis=1)[0]
        prediction = pred_classes[pred]
        
        return jsonify({'prediction_result': prediction})
    
    except Exception as e:
        return jsonify({'error': 'Given image can not be recognized.'}), 500
    
@app.route("/api/login",  methods=['POST'])

def login():
    if request.method == 'POST' and 'cs_user_name' in request.form and 'cs_user_password' in request.form:
        user_name = request.form['cs_user_name']
        user_password = request.form['cs_user_password']

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
   
        cursor.execute('SELECT * FROM users WHERE user_name = %s', (user_name,))
        account = cursor.fetchone()
        if account and bcrypt.checkpw(user_password.encode('utf-8'), account['user_password'].encode('utf-8')):
       
                access_token = create_access_token(identity=user_name)
                msg = 'Logged in successfully!'
                return jsonify({"access_token": access_token}), 200
 
        else:
            msg = 'Incorrect username/password!'
            response = jsonify({'error': msg})
            response.status_code = 400
            return response

    else:
        response = jsonify({'error': 'Invalid request method.'})
        response.status_code = 405
        return response
    
@app.route("/api/create_account", methods=['GET', 'POST'])

def createAccount():

    if request.method == 'POST' and 'cs_user_name' in request.form and 'cs_user_password' in request.form and 'cs_user_email' in request.form and 'cs_user_full_name' in request.form:
         
        user_full_name = request.form['cs_user_full_name']
        user_name = request.form['cs_user_name']
        user_email = request.form['cs_user_email']
        password = request.form['cs_user_password']

        # Hash the password using bcrypt
        salt = bcrypt.gensalt()
        user_password = bcrypt.hashpw(password.encode('utf-8'), salt)

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    
        cursor.execute('SELECT * FROM users WHERE user_name = %s', (user_name,))
        account = cursor.fetchone()

        cursor.execute('SELECT * FROM users WHERE user_email = %s', (user_email,))
        check_email = cursor.fetchone() 

        created_at = datetime.datetime.now()

        if check_email and account:
            msg = 'Acount already exists!'
            response = jsonify({'error': msg})
            response.status_code = 409 

        elif check_email:
            msg = 'Email already exists!'
            response = jsonify({'error': msg})
            response.status_code = 409

        elif account:
            msg = 'User name already exists!'
            response = jsonify({'error': msg})
            response.status_code = 409
  
        elif not re.match(r'[^@]+@[^@]+\.[^@]+', user_email):
            msg = 'Invalid email address!'
            response = jsonify({'error': msg})
            response.status_code = 400

        elif not user_name or not user_password or not user_email or not user_full_name:
            msg = 'Please fill out the form!'
            response = jsonify({'error': msg})
            response.status_code = 400

        else:
            my_cursor  = mysql.connection.cursor()
            my_cursor.execute('INSERT INTO users (user_name, user_password, user_email, user_full_name,created_at) VALUES (%s, %s, %s, %s, %s)', (user_name, user_password, user_email, user_full_name,created_at))

            mysql.connection.commit()
            msg = 'You have successfully registered!'
            response = jsonify({'success': msg})
            response.status_code = 201

        return response

    elif request.method == 'POST':
        msg = 'Please fill out the form!'
        response = jsonify({'error': msg})
        response.status_code = 400
        return response

    else:
        response = jsonify({'error': 'Invalid request method.'})
        response.status_code = 405
        return response
    

@app.route("/api/checkUser", methods=['GET', 'POST'])

def checkUser():
    if request.method == 'POST' and 'cs_user_name' in request.form and 'cs_user_email' in request.form:
        user_name = request.form['cs_user_name']
        user_email = request.form['cs_user_email']

        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
   
        cursor.execute('SELECT * FROM users WHERE user_name = %s AND user_email = %s', (user_name, user_email,))

        account = cursor.fetchone()
        if account:   
            msg = 'successfully!'
            response = jsonify({'user_name': user_name, 'user_email': user_email, 'message': msg})
            response.status_code = 200
            return response

 
        else:
            msg = 'Incorrect username/email!'
            response = jsonify({'error': msg})
            response.status_code = 400
            return response

    else:
        response = jsonify({'error': 'Invalid request method.'})
        response.status_code = 405
        return response


@app.route("/api/changePassword", methods=['GET', 'POST'])

def changePassword():
    if request.method == 'POST' and 'cs_user_name' in request.form and 'cs_password' in request.form:
        user_name = request.form['cs_user_name']
        user_email = request.form['cs_user_email']
        user_password = request.form['cs_password']
        user_conform_password = request.form['cs_conform_password']

        if user_password != user_conform_password:
            msg = 'Passwords do not match!'
            response = jsonify({'error': msg})
            response.status_code = 400
            return response
        else:
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)

            salt = bcrypt.gensalt()
            password = bcrypt.hashpw(user_password.encode('utf-8'), salt)

            cursor.execute('UPDATE users SET user_password = %s WHERE user_name = %s', (password, user_name))
            mysql.connection.commit()
            msg = 'Password updated successfully!'
            response = jsonify({'user_name': user_name, 'message': msg})
            response.status_code = 200
            return response
    
    else:
        response = jsonify({'error': 'Invalid request method.'})
        response.status_code = 405
        return response    

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
